In [1]:
import sys, os, json, random
import numpy as np
import tensorflow as tf
import pandas as pd

In [0]:
#company names
#dataset = pd.read_csv('https://raw.githubusercontent.com/EliotAndres/char-rnn-tensorflow-js/master/python/companies.csv', header=None)
#dataset = set(str(v) for v in dataset.to_numpy().squeeze())

# baby names
#dataset = pd.read_csv('https://raw.githubusercontent.com/dongjason1/python_project_showcase/master/data/NationalNames.csv')
#dataset = set(str(v) for v in dataset[[1]].to_numpy().squeeze())

# college names
# dataset = pd.read_csv('https://raw.githubusercontent.com/dongjason1/python_project_showcase/master/data/College.csv')
# dataset = set(str(v) for v in dataset[['Unnamed: 0']].to_numpy().squeeze())

# benedict cumberbatch 
# future work: https://the-toast.net/2013/12/02/a-linguist-explains-the-rules-of-summoning-benedict-cumberbatch/
dataset = pd.read_csv('https://raw.githubusercontent.com/dongjason1/python_project_showcase/master/data/benedict_cumberbatch.csv', header=None)
dataset = set(str(v) for v in dataset.to_numpy().squeeze())

In [3]:
training_data = '\n'.join(dataset)
# Get chars and map to int
chars = sorted(list(set(training_data)))
char_to_int = dict((c,i) for i,c in enumerate(chars))
int_to_char = dict((i,c) for i,c in enumerate(chars))
print('# Names: {}'.format(len(dataset)))
print('Number of Chars in Training Data: {}'.format(len(training_data)))
print('Number of Unique Chars: {}'.format(len(chars)))

# Names: 537
Number of Chars in Training Data: 12036
Number of Unique Chars: 56


In [4]:
from tensorflow.keras.layers import LSTM, Dense, Activation
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import RMSprop

# Model Definition
hidden_units = 64
data_length = 10
step = 3
model = Sequential()
model.add(LSTM(hidden_units, input_shape=(data_length, len(chars))))
model.add(Dense(units=len(chars)))
model.add(Activation('softmax'))
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer)

W1029 04:15:42.476196 140054744962944 deprecation.py:506] From /usr/local/lib/python2.7/dist-packages/tensorflow_core/python/ops/resource_variable_ops.py:1630: calling __init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [0]:
data_length = 10
step = 3
# Split into feature and target
inp = []
out = []
for i in range(0, len(training_data) - data_length, step):
    inp.append(training_data[i:i+data_length])
    out.append(training_data[i+data_length])

# Vectorize
x = np.zeros((len(inp), data_length, len(chars)), dtype=np.bool)
y = np.zeros((len(out), len(chars)), dtype=np.bool)
for i, sequence in enumerate(inp):
    for t, char in enumerate(sequence):
        x[i, t, char_to_int[char]] = 1
    y[i, char_to_int[out[i]]] = 1

In [6]:
print("{}: Number of Documents in Batch: {}".format(i, len(x)))
model.fit(x, y, epochs=2, verbose=1)

W1029 04:15:42.948724 140054744962944 deprecation.py:323] From /usr/local/lib/python2.7/dist-packages/tensorflow_core/python/ops/math_grad.py:1424: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


4008: Number of Documents in Batch: 4009
Train on 4009 samples
Epoch 1/2
4009/4009 [==============================] - 3s 674us/sample - loss: 2.5763
Epoch 2/2
4009/4009 [==============================] - 2s 424us/sample - loss: 1.8385


In [0]:
# Gets a random seed
def get_seed():
    seed = ""
    for i in range(10):
        num = random.randint(0,len(chars))
        seed += int_to_char[num]
    return seed

# Samples index from given distribution and temperature
def sample(preds, temp):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temp
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)
    
# One hot encodes a string
def string_to_vec(string):
    vec = np.zeros((1, len(string), len(chars)), dtype=np.bool)
    for t, char in enumerate(string):
        vec[0, t, char_to_int[char]] = 1
    return vec

# generates a list of names from the model and an initial seed
def generate_name(model, seed, num_results=10, temp = 0.7):
    generated = []
    text = seed
    word = seed
    while num_results>0:
        vec = string_to_vec(text)
        preds = model.predict(vec)[0]
        predicted = sample(preds, temp)
        character = int_to_char[predicted]
        text = text[1:] + character
        word += character
        if character == '\n':
            num_results -=1
            try:
              generated.append(word[:-1].encode('ascii'))
            except:
              print(word[:-1])
            word = ""
        
    return generated

In [14]:
# Let's see some results
result = generate_name(model, get_seed(), 15, 0.5)
result

fbFn�rsvY�t Cucumberbuck


['Bendydick Cucumberbatch',
 'Bundybink Cumberbuck',
 'Benadick Cumberbatch',
 'Benadica Cumbenbunk',
 'Bundybick Cumbindensn',
 'Bendiblick Cucunumber',
 'Buncersnatch',
 'Bendydick Cumberbunt',
 'Bennyeolen Cumbersnatch',
 'Bunderbu Cumberbutch',
 'Bumberbink Cumsnatch',
 'Bendydick Cumberbonch',
 'Bendydick Cumberbonch',
 'Benedick Cumbersnatch']

In [0]:
for r in result:
  if r in dataset:
    print(r)